In [17]:
import gdspy
import uuid

def create_resonator_with_border_and_hole(side_length, border_width, hole_length, hole_position):
    cell_name = str(uuid.uuid4())  # Generate unique name for the cell
    cell = gdspy.Cell(cell_name)
    
    # Create the outer square border
    outer_square = gdspy.Rectangle((-side_length/2 - border_width, -side_length/2 - border_width),
                                    (side_length/2 + border_width, side_length/2 + border_width),
                                    layer=0)
    
    # Create the inner square resonator
    inner_square = gdspy.Rectangle((-side_length/2, -side_length/2), (side_length/2, side_length/2), layer=0)
    
    # Subtract the inner square from the outer square
    outer_square = gdspy.boolean(outer_square, inner_square, 'not', layer=0)
    
    # Add the outer square to the cell
    # cell.add(outer_square)
    
    # Create the hole at specified position
    if hole_position.lower() == 'left':
        hole = gdspy.Rectangle((-side_length/2 - border_width/2 - hole_length/2, -border_width/2),
                               (-side_length/2 - border_width/2 + hole_length/2, border_width/2),
                               layer=1)
    elif hole_position.lower() == 'right':
        hole = gdspy.Rectangle((side_length/2 + border_width/2 - hole_length/2, -border_width/2),
                               (side_length/2 + border_width/2 + hole_length/2, border_width/2),
                               layer=1)
    elif hole_position.lower() == 'top':
        hole = gdspy.Rectangle((-border_width/2, side_length/2 + border_width/2 - hole_length/2),
                               (border_width/2, side_length/2 + border_width/2 + hole_length/2),
                               layer=1)
    elif hole_position.lower() == 'bottom':
        hole = gdspy.Rectangle((-border_width/2, -side_length/2 - border_width/2 - hole_length/2),
                               (border_width/2, -side_length/2 - border_width/2 + hole_length/2),
                               layer=1)
    else:
        raise ValueError("Invalid hole position. Choose from 'left', 'right', 'top', or 'bottom'.")
    
    # Add the hole to the cell
    outer_square_hole=gdspy.boolean(outer_square,hole,'not',layer=0)
    cell.add(outer_square_hole)
    
    return cell

# Set parameters
side_length = 10        # Length of the square resonator side
border_width = 1       # Width of the border around the resonator
hole_length = 3        # Length of the hole
hole_position = 'right'  # Position of the hole: 'left', 'right', 'top', or 'bottom'

# Create the resonator with border and hole
resonator_with_border_and_hole = create_resonator_with_border_and_hole(side_length, border_width, hole_length, hole_position)

# Create the library
library = gdspy.GdsLibrary()
library.add(resonator_with_border_and_hole)

# Save library to a GDSII file
library.write_gds('gdsfile/resonator_with_border_and_hole.gds')


In [23]:
import random
# Create an array of resonators with border and hole
array_cell_name = str(uuid.uuid4())  # Generate unique name for the cell
array_cell = gdspy.Cell(array_cell_name)
array_dist = 15
for i in range(4):
    for j in range(4):
        x_offset = array_dist * i
        y_offset = array_dist * j
        hole_position = random.choice(['left', 'right', 'top', 'bottom'])  # Randomly select hole position
        resonator_with_border_and_hole = create_resonator_with_border_and_hole(side_length, border_width, hole_length, hole_position)
        array_cell.add(gdspy.CellReference(resonator_with_border_and_hole, (x_offset, y_offset)))

# Create the library
library = gdspy.GdsLibrary()
library.add(array_cell)

# Save library to a GDSII file
library.write_gds('gdsfile/resonator_with_border_and_hole_array_random.gds')